<a href="https://colab.research.google.com/github/yatora200403/Konverter-Suhu/blob/main/crop_yield_with_fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd
import json

In [12]:
# fungsi logika fuzzy untuk prediksi hasil panen
def fuzzy_yield_prediction(land_area, rainfall, disease_level, temperature, planting_distance):
    #fuzzy untuk curah hujan
    if rainfall == "sangat rendah":
        rainfall_factor = 0.3
    elif rainfall == "rendah":
        rainfall_factor = 0.7
    elif rainfall == "normal":
        rainfall_factor = 1.0
    elif rainfall == "tinggi":
        rainfall_factor = 0.8
    else:
        rainfall_factor = 0.4

    #fuzzy untuk tingkat penyakit
    if disease_level == "sangat rendah":
        disease_factor = 1.0
    elif disease_level == "rendah":
        disease_factor = 0.8
    elif disease_level == "normal":
        disease_factor = 0.6
    elif disease_level == "tinggi":
        disease_factor = 0.2
    else:
        disease_factor = 0.05

    #efek jarak tanam terhadap hasil panen
    if planting_distance == "20cmx20cm":
        planting_factor = 1.2
    elif planting_distance == "25cmx25cm":
        planting_factor = 1.0
    elif planting_distance == "30cmx30cm":
        planting_factor = 0.9


    #hasil panen dalam kg berdasarkan faktor fuzzy
    base_yield_per_m2 = 8 * 1000 / 10000  # 8 ton per hektar = 8000 kg per 10,000 m2
    yield_prediction = (
        land_area * base_yield_per_m2 * rainfall_factor * disease_factor * planting_factor * (30 - abs(temperature - 25)) / 30
    )

    return round(yield_prediction, 2)

- GKP (Gabah Kering Panen) = 100% dari hasil panen di sawah
- GKG (Gabah Kering Giling) = 70% dari GKP
- Beras = 65% dari GKG

In [ ]:
#fungsi untuk menghitung jumlah bibit ideal
def seed_amount(land_area, planting_distance):
    if planting_distance == "20cmx20cm":
        seeds_per_m2 = 25
    elif planting_distance == "25cmx25cm":
        seeds_per_m2 = 16
    elif planting_distance == "30cmx30cm":
        seeds_per_m2 = 11

    base_seed_per_m2 = seeds_per_m2 / 1000
    variation = np.random.uniform(0.9, 1.1)
    total_seed = land_area * base_seed_per_m2 * variation
    return round(total_seed, 2)

In [14]:
# data untuk curah hujan, tingkat penyakit, dan jarak tanam
rainfall_levels = ["sangat rendah", "rendah", "normal", "tinggi", "sangat tinggi"]
disease_levels = ["sangat rendah", "rendah", "normal", "tinggi", "sangat tinggi"]
planting_distances = ["20cmx20cm", "25cmx25cm", "30cmx30cm"]

data = []

land_area = int(input("luas lahan : "))  #luas lahan
rainfall = str(input("curah hujan : "))  #curah hujan
rainfall = rainfall.lower()
disease_level = str(input("tingkat penyakit : "))  #tingkat penyakit
disease_level = disease_level.lower()
temperature = float(input("temperature C: "))  #suhu rata-rata(C)
planting_distance = input("jarak tanam : ")  #jarak tanam
planting_distance = planting_distance.lower()
seed_weight = seed_amount(land_area, planting_distance)  #jumlah bibit padi (kg)

gkp = fuzzy_yield_prediction(land_area, rainfall, disease_level, temperature, planting_distance)
gkg = round(gkp * 0.7, 2)  #Gabah Kering Giling = 70% dari GKP
rice = round(gkg * 0.65, 2)  #Beras = 65% dari GKG

data.append([land_area, rainfall, disease_level, temperature, planting_distance, seed_weight, gkp, gkg, rice])

columns = [
    "Luas Lahan", #m2
    "Curah Hujan",
    "Tingkat Penyakit",
    "Suhu", #C
    "Jarak Tanam", #cm
    "Jumlah Bibit", #kg
    "GKP", #kg
    "GKG", #kg
    "Beras" #kg
]
dataset = pd.DataFrame(data, columns=columns)
json_response = dataset.to_json(orient="records")
print(json_response)

# dataset.to_csv("crop yield.csv", index=False)
# print("Dataset time series berhasil dibuat dan disimpan sebagai 'crop yield.csv'.")

luas lahan : 1000
curah hujan : normal
tingkat penyakit : normal
temperature C: 21.8
jarak tanam : 20cmx20cm
[{"Luas Lahan":1000,"Curah Hujan":"normal","Tingkat Penyakit":"normal","Suhu":21.8,"Jarak Tanam":"20cmx20cm","Jumlah Bibit":22.56,"GKP":514.56,"GKG":360.19,"Beras":234.12}]
